In [1]:
import torch
import numpy as np
import transforms_data as td
from PIL import Image
import glob


CATEGORIES = ["Falco", "Jesse", "Konrad", "Nils", "Johannes"]
CATEGORIES.sort()
AUTHORIZED = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"

model = torch.load('./class_sample/model.pt')

In [2]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test((RESIZE_Y, RESIZE_X),[0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    image_array.append(image_transformed.type('torch.FloatTensor'))
    #image_array.append(image_transformed.type('torch.cuda.FloatTensor'))


In [3]:
all_classes = []
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)


['0.053112540394067764' '0.049723513424396515' '0.4816586375236511'
 '0.28473031520843506' '0.13077504932880402' '2.0' 'Johannes'] 

['0.040945570915937424' '0.03889249265193939' '0.7236711382865906'
 '0.15605700016021729' '0.04043375328183174' '2.0' 'Johannes'] 

['0.04938454553484917' '0.039005402475595474' '0.7324987649917603'
 '0.14009550213813782' '0.03901584446430206' '2.0' 'Johannes'] 

['0.02913811057806015' '0.03202831372618675' '0.7521273493766785'
 '0.1299075335264206' '0.05679870396852493' '2.0' 'Johannes'] 

['0.028008174151182175' '0.022685419768095016' '0.7349361777305603'
 '0.1204887330532074' '0.09388146549463272' '2.0' 'Johannes'] 

['0.10866987705230713' '0.10804121196269989' '0.5482330322265625'
 '0.1292331963777542' '0.10582271963357925' '2.0' 'Johannes'] 

['0.17764228582382202' '0.116679847240448' '0.3287028968334198'
 '0.24415956437587738' '0.132815420627594' '2.0' 'Johannes'] 

['0.15601035952568054' '0.06867317855358124' '0.442903071641922'
 '0.203803777694702

In [4]:
# Hier besser die Warscheinlichkeit über alle Bilder ermitteln und darüber prüfen.
# Beispiel: Bei 5 Bilder muss die aufsummierte Wahrscheinlichkeit für eine Person >4 sein!!

NUMBER_AUTHORIZED = int(.8*len(image_array))
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in AUTHORIZED and authentification_dict[a] >= NUMBER_AUTHORIZED:
        print("Access granted! Welcome "  + a + "!")
    else:
        print("Access denied")
        break


{'Johannes': 8}
Access denied
